In [1]:
import re

def words(text): 
  return re.findall(r'\w+', text.lower())

In [3]:
# test

open('big.txt').read() # 把整個txt檔的內容變成一個大字串
words(open('big.txt').read()) # 把大字串變成一個單字一個單字(都小寫)組成的一維陣列

str

In [5]:
from collections import Counter

word_count = Counter(words(open('big.txt').read()))
word_count # 每個單字出現的數量，以字典的形式呈現

In [35]:
def percent(word): 
  sum_of_word_i = word_count[word] # 某個單字出現的次數
  sum_of_all_words = sum(word_count.values()) # 所有單字出現的次數
  return sum_of_word_i / sum_of_all_words # 某個單字在整篇文章中所佔的比例

In [7]:
# test
def edits1(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    return splits

edits1('apple')

[('', 'apple'),
 ('a', 'pple'),
 ('ap', 'ple'),
 ('app', 'le'),
 ('appl', 'e'),
 ('apple', '')]

In [10]:
# test
def edits1(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    # 抓 少打一個字:
    deletes    = [L + R[1:]               for L, R in splits if R]
    return deletes

edits1('apple')

['pple', 'aple', 'aple', 'appe', 'appl']

In [11]:
# test
def edits1(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    # 抓 其中兩個字母打相反:
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    return transposes

edits1('apple')

['paple', 'apple', 'aplpe', 'appel']

In [13]:
# test
def edits1(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    # 抓 若某字母不小心打兩次:
    doubles    = [L + R[0] + R         for L, R in splits if R]
    return doubles 

edits1('apple')

['aapple', 'appple', 'appple', 'applle', 'applee']

In [38]:
letters  = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
    # 單字分屍
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]

    # 抓 少打一個字:
    deletes    = [L + R[1:]               for L, R in splits if R]
    # 抓 其中兩個字母打相反:
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    # 抓 各個字母打錯的情況:
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    # 抓 字母之間的每個空隙多打一個字母的情況:
    inserts    = [L + c + R               for L, R in splits for c in letters]
    # 抓 若某字母不小心打兩次:
    doubles    = [L + R[0] + R         for L, R in splits if R]
    # 抓 連續字母少打一個字的情況:
    one = [L + R[1:] for L, R in splits if len(R)>1 and R[0]==R[1]]
    return set(deletes + transposes + replaces + inserts + doubles + one)

In [ ]:
# test

print(edits1('apple'), "\n")

for item in edits1('apple'):
    print(item)

In [ ]:
# test

def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

for item in edits2('apple'):
    print(item)

In [29]:
# word的所有情況再各自跑出所有情況
def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
    # 先寫的for會在外圈

# 某個情況剛好是字典裡的單字 (有被word_count統計到)，把他們搜集出來並回傳
def known(words): 
    return set(w for w in words if w in word_count)

In [33]:
# 回傳的優先順序應是由左至右 (1.沒有單字才回傳2.，以此類推)
def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    # 1. known([word])
    # 2. known(edits1(word))
    # 3. known(edits2(word))
    # 4. [word]

def correction(word): 
    return max(candidates(word), key=percent)
    # candidates(word)的結果應該是1.2.3.4.某一個set，裡面有很多個單字
    # key=percent，就是把每個單字的出現比例算出來，然後用max回傳出現比例最高的那個單字

In [36]:
# test

correction('apple')

'apple'

In [ ]:
choice = open('output.txt').read()
choice = choice.split('\n')

option_a = st.selectbox('Choose a word or ... ',choice)
option_b = st.text_input('or type your own!!')

result = st.sidebar.checkbox('Show original word')
def run(original_word):
  if result == True:
    st.write('Original word: *%s*'%original_word)
  cw = correction(original_word)
  if original_word == cw:
    st.success("--- *%s* --- is the correct spelling!"%original_word)
  else:
    st.error("Correction: *%s*"%cw)

if option_b != '' and option_a != ' ':
  original_word = option_b
  run(original_word)

if option_b != '' and option_a == ' ':
  original_word = option_b
  run(original_word)

if option_b == '' and option_a != ' ':
  original_word = option_a
  run(original_word)